In [1]:
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
raw_data = pd.read_csv(os.path.join("Resources", "Data", "growth_designation.csv"))
raw_data

,ColonyCount,TotalProduction,YieldPerColony,PricePerLB,ProductionValue,Stocks,Clothianidin,Imidacloprid,Thiamethoxam,Acetamiprid,Thiacloprid,CombinedNeonic,GrowthOutcome
0,16000.0,928000.0,58,0.69,640000.0,28000.0,0.0,716.5,0.0,0.0,0.0,716.5,0
1,15000.0,960000.0,64,0.87,835000.0,96000.0,0.0,371.6,0.0,0.0,0.0,371.6,0
2,14000.0,924000.0,66,0.81,748000.0,92000.0,0.0,6704.8,0.0,0.0,0.0,6704.8,0
3,16000.0,1136000.0,71,0.72,818000.0,159000.0,0.0,1836.3,0.0,0.0,0.0,1836.3,1
4,17000.0,1156000.0,68,0.56,647000.0,185000.0,0.0,1251.2,0.0,0.0,0.0,1251.2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,50000.0,2550000.0,51,1.87,4769000.0,459000.0,1174.9,277.0,423.3,0.0,0.0,1875.2,1
891,47000.0,3102000.0,66,2.11,6545000.0,558000.0,840.9,155.9,526.2,0.0,0.0,1523.0,0
892,38000.0,2318000.0,61,2.08,4821000.0,255000.0,1262.0,258.2,566.1,0.0,0.0,2086.3,0
893,38000.0,2926000.0,77,1.90,5559000.0,146000.0,0.0,114.4,19.0,0.0,0.0,133.4,0


In [4]:
target = raw_data['GrowthOutcome']
target_names = ['Growth', 'No-Growth']

In [5]:
X = raw_data.drop(columns=['GrowthOutcome'])
y = raw_data["GrowthOutcome"]
feature_names = X.columns
print(X.shape, y.shape)

(895, 12) (895,)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train

,ColonyCount,TotalProduction,YieldPerColony,PricePerLB,ProductionValue,Stocks,Clothianidin,Imidacloprid,Thiamethoxam,Acetamiprid,Thiacloprid,CombinedNeonic
705,7000.0,385000.0,55,1.67,643000.0,92000.0,698.2,12633.2,4017.1,695.6,16.0,18060.1
453,11000.0,539000.0,49,1.43,771000.0,243000.0,0.0,2145.2,0.0,0.0,0.0,2145.2
289,48000.0,5472000.0,114,0.49,2681000.0,821000.0,0.0,628.4,0.0,0.0,0.0,628.4
798,5000.0,245000.0,49,2.01,492000.0,69000.0,229.7,85.6,146.8,22.8,149.8,634.7
35,24000.0,1368000.0,57,1.42,1943000.0,301000.0,4346.9,10079.4,21186.5,349.1,0.0,35961.9
...,...,...,...,...,...,...,...,...,...,...,...,...
715,7000.0,413000.0,59,4.04,1669000.0,78000.0,958.2,5303.8,2472.5,580.7,11.3,9326.5
767,7000.0,294000.0,42,1.50,441000.0,103000.0,0.0,1660.0,0.0,0.0,0.0,1660.0
72,470000.0,23500000.0,50,1.32,31020000.0,3525000.0,0.0,36653.3,5021.8,3007.5,0.0,44682.6
235,8000.0,408000.0,51,2.14,873000.0,147000.0,56387.8,53736.8,15199.8,45.6,4.3,125374.3


In [7]:
X_test

,ColonyCount,TotalProduction,YieldPerColony,PricePerLB,ProductionValue,Stocks,Clothianidin,Imidacloprid,Thiamethoxam,Acetamiprid,Thiacloprid,CombinedNeonic
535,15000.0,1155000.0,77,0.91,1051000.0,219000.0,0.0,1547.7,0.0,0.0,0.0,1547.7
34,28000.0,2100000.0,75,1.35,2835000.0,525000.0,2168.9,9093.8,25102.8,995.7,0.0,37361.2
375,128000.0,8448000.0,66,1.55,13094000.0,1774000.0,45267.0,40330.4,23717.8,420.8,0.0,109736.0
81,370000.0,17760000.0,48,1.65,29304000.0,3730000.0,14229.5,112450.6,7391.2,15457.0,0.0,149528.3
6,14000.0,1022000.0,73,0.72,736000.0,235000.0,0.0,1490.8,0.0,0.0,0.0,1490.8
...,...,...,...,...,...,...,...,...,...,...,...,...
546,6000.0,354000.0,59,1.42,503000.0,138000.0,478.1,3233.7,270.8,46.3,0.0,4028.9
509,50000.0,3750000.0,75,2.04,7650000.0,1688000.0,135037.2,39838.2,51112.9,1.0,0.0,225989.3
331,5000.0,205000.0,41,2.05,420000.0,33000.0,140.3,2738.7,1302.1,395.4,0.0,4576.5
888,34000.0,1224000.0,36,1.59,1946000.0,282000.0,756.3,128.1,663.7,0.0,0.0,1548.1


In [8]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.5

In [9]:
rf = RandomForestClassifier(n_estimators=800)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.6339285714285714

In [10]:
feature_importance = sorted(zip(rf.feature_importances_, feature_names), reverse=True)
feature_importance

[(0.11462629344887966, 'ProductionValue'),
 (0.10470393270785466, 'TotalProduction'),
 (0.10402035769094352, 'Stocks'),
 (0.10352687445075151, 'Imidacloprid'),
 (0.10330932384610732, 'PricePerLB'),
 (0.09795241378236108, 'CombinedNeonic'),
 (0.09350587276585523, 'YieldPerColony'),
 (0.08768855310316245, 'ColonyCount'),
 (0.07044564858483048, 'Thiamethoxam'),
 (0.051304971329463385, 'Clothianidin'),
 (0.048923446796333744, 'Acetamiprid'),
 (0.01999231149345707, 'Thiacloprid')]

In [11]:
feats = {}
for importance, feature in feature_importance:
    feats[feature] = importance
    
# feats

rf_importance_df = pd.DataFrame.from_dict(feats)
rf_importance_df

ValueError: If using all scalar values, you must pass an index

In [12]:
from sklearn.metrics import classification_report
honeybee_prediction = rf.predict(X_test)
print(classification_report(y_test, honeybee_prediction))

              precision    recall  f1-score   support

           0       0.65      0.87      0.75       139
           1       0.54      0.25      0.34        85

    accuracy                           0.63       224
   macro avg       0.60      0.56      0.54       224
weighted avg       0.61      0.63      0.59       224

